### Quick Look

In [1]:
import os
baudrillard = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/simulacra-and-simulation.txt'
baudrillard_2 = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/simulations.txt'
def load_data(path: str) -> str:

    text_file = os.path.join(path)
    with open(text_file, "r") as f:
        data = f.read()

    return data

_data = load_data(baudrillard)
_data_2 = load_data(baudrillard_2)
main_data = _data + _data_2

In [2]:
import numpy as np

view_line_range = (100, 150)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in main_data.split()})))

lines = main_data.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(main_data.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 14440
Number of lines: 11204
Average number of words in each line: 7.909139593002499

The lines 100 to 150:
whose operation is nuclear and genetic, no longer at all specular or discursive. It is all of 
metaphysics that is lost. No more mirror of being and appearances, of the real and its 
concept. No more imaginary coextensivity: it is genetic miniaturization that is the 
dimension of simulation. The real is produced from miniaturized cells, matrices, and 
memory banks, models of control - and it can be reproduced an indefinite number of 
times from these. It no longer needs to be rational, because it no longer measures itself 
against either an ideal or negative instance. It is no longer anything but operational. In 
fact, it is no longer really the real, because no imaginary envelops it anymore. It is a 
hyperreal, produced from a radiating synthesis of combinatory models in a hyperspace 
without atmosphere. 

By crossing into a spac

### More Developed Dive

In [3]:
from utils import data

baudrillard = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/simulacra-and-simulation.txt'
baudrillard_2 = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/simulations.txt'

text = data.convertManyToOne([baudrillard, baudrillard_2])

tmp_path = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/tmp'
tmp_file = 'tmp_combined.txt'

if not os.path.exists(tmp_path):
    os.mkdir(tmp_path)

with open(f'{tmp_path}/{tmp_file}', 'w') as f:

    f.write(text)
    f.close()

data.preprocess_and_save(
    text_path=f'{tmp_path}/{tmp_file}',
    text_title='baudrillard_combined'
)
int_text, w2i, i2w, token_dict = data.load_preprocess('baudrillard_combined')

In [4]:
import torch

train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('CPU')

CPU


In [5]:
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

def batch_data(
    words: list, 
    sequence_length: int, 
    batch_size: int) -> DataLoader:

    words = np.array(words)
    total_batch_size = batch_size * sequence_length
    n_batches = len(words) // batch_size

    words = words[:n_batches * total_batch_size]
    feature, target = [], []

    target_words_length = words[:-sequence_length]
    for idx in range(0, len(target_words_length)):

        feature.append(words[idx: idx + sequence_length])
        target.append(words[idx + sequence_length])

    batch_nums = len(words) // batch_size
    feature = feature[:batch_nums * batch_size]
    target = target[:batch_nums * batch_size]

    feature_tensors = torch.from_numpy(np.asarray(feature))
    target_tensors = torch.from_numpy(np.asarray(target))

    data_set = TensorDataset(feature_tensors, target_tensors)
    data_loader = DataLoader(data_set, batch_size=batch_size, shuffle=True)

    return data_loader

In [6]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape) # batch_size x sequence_length
print(sample_x) # batch
print()
print(sample_y.shape) # text | len(words)
print(sample_y) # text | list(int)

torch.Size([10, 5])
tensor([[22, 23, 24, 25, 26],
        [42, 43, 44, 45, 46],
        [32, 33, 34, 35, 36],
        [44, 45, 46, 47, 48],
        [20, 21, 22, 23, 24],
        [39, 40, 41, 42, 43],
        [10, 11, 12, 13, 14],
        [41, 42, 43, 44, 45],
        [33, 34, 35, 36, 37],
        [31, 32, 33, 34, 35]])

torch.Size([10])
tensor([27, 47, 37, 49, 25, 44, 15, 46, 38, 36])


In [7]:
n_loader = batch_data(int_text, sequence_length=7, batch_size=32)
data_iter = iter(n_loader)
x,y = data_iter.next()

print(x)
print(y)

tensor([[ 3220,     9,  9090,  1349,   610,    58,     3],
        [    7,    32,    21,  2963,    33,  2964,  1392],
        [  118,     6,     0,   134,     1,     3,  1693],
        [    0,   293,   380,     8,  2100,    14,    20],
        [   19, 11258,     3,   226,     1,  5594,     1],
        [ 5346,     0,  1738,     1,     0,  2281,  1456],
        [    4,    49,     0,  5584, 11218,    16,  2061],
        [  249,   252,    40,    91,   252, 16166,    22],
        [10739,   115,    46,    41,    93,   119,     3],
        [    2,   306,    29,   147,     5, 10760,     0],
        [14500,    74,     7, 14501,  1637, 14502,   540],
        [    1,  5880,    79, 13638,  4487,    37,    49],
        [  894,    30,    17,     0,   685,     1, 12011],
        [ 1320,     4,   639,  3317,    22,     0,   329],
        [  389,     0,  3306, 11114,     0, 11115,   283],
        [    5,     0,  5932, 13843,     1,   671,     6],
        [    0, 11535,  5637,     0, 11536, 11537,  2131

In [8]:
from LSTM import LSTM

In [9]:
import torch
import torch.nn as nn

def forward_back_prop(
    model, 
    optimizer, 
    criterion, 
    inp, 
    target, 
    hidden,
    train_on_gpu=False):

    if train_on_gpu:
        inp, target = inp.cuda(), target.cuda()

    hidden = tuple([each.data for each in hidden])

    model.zero_grad()
    output, hidden = model(inp, hidden)

    loss = criterion(output, target)
    loss.backward()

    clip = 5
    nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()

    return loss.item(), hidden
    

    

In [10]:
def train_LSTM(
    model,
    batch_size,
    optimizer,
    criterion,
    n_epochs,
    train_loader,
    show_every_n_batches=100):

    batch_losses = []

    model.train()

    print(f' | Training for {n_epochs} epoch(s).....')
    for e in range(1, n_epochs + 1):

        hidden = model.init_hidden(batch_size)

        for batch_idx, (inputs, labels) in enumerate(train_loader, 1):

            n_batches = len(train_loader.dataset)//batch_size

            if batch_idx > n_batches:
                break

            loss, hidden = forward_back_prop(
                                model=model,
                                optimizer=optimizer,
                                criterion=criterion,
                                inp=inputs,
                                target=labels,
                                hidden=hidden
            )
            batch_losses.append(loss)

            if batch_idx % show_every_n_batches == 0:
                print(f' | EPOCH: {e:>4}/{n_epochs:>4} -- LOSS: {np.average(batch_losses)}')
                batch_losses = []

    return model



In [11]:
num_epochs = 25
learning_rate = 0.0001
vocab_size = len(w2i)
output_size = vocab_size
embedding_dim = 400
hidden_dim = 512
n_layers = 3
sequence_length = 7
batch_size = 32
show_every_n_batches = 500

In [12]:

model = LSTM(
            vocab_size=vocab_size,
            output_size=output_size,
            embedding_dim=embedding_dim,
            hidden_dim=hidden_dim,
            n_layers=n_layers
)

if train_on_gpu:
    model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()


In [13]:
trained_model = train_LSTM(
                    model=model,
                    batch_size=batch_size,
                    optimizer=optimizer,
                    criterion=criterion,
                    n_epochs=num_epochs,
                    train_loader=n_loader,
                    show_every_n_batches=show_every_n_batches
)

 | Training for 25 epoch(s).....


In [ ]:
data.save_model(filename='/models/baudrillard_v1', decoder=trained_model)